In [1]:
import pandas as pd
import pymongo
import requests

In [2]:
#!pip install kaggle

In [3]:
Population = pd.read_csv("resources/population_by_zip_2010.csv")
Population
pop_df = Population.drop(columns=["geo_id"]).rename(columns={"zipcode":"ZipCode"})
pop_df

,population,minimum_age,maximum_age,gender,ZipCode
0,50,30.0,34.0,female,61747
1,5,85.0,NaN,male,64120
2,1389,30.0,34.0,male,95117
3,231,60.0,61.0,female,74074
4,56,0.0,4.0,female,58042
...,...,...,...,...,...
1048570,83,25.0,29.0,male,65705
1048571,53,65.0,66.0,female,18074
1048572,0,18.0,19.0,female,50593
1048573,37,0.0,4.0,male,74957


In [4]:
inventory = pd.read_csv("resources/RDC_InventoryCoreMetrics_Zip_Hist.csv")
inventory



C:\Users\glaud\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Month,ZipCode,ZipName,Footnote,Median Listing Price,Median Listing Price M/M,Median Listing Price Y/Y,Active Listing Count,Active Listing Count M/M,Active Listing Count Y/Y,...,Pending Listing Count Y/Y,Avg Listing Price,Avg Listing Price M/M,Avg Listing Price Y/Y,Total Listing Count,Total Listing Count M/M,Total Listing Count Y/Y,Pending Ratio,Pending Ratio M/M,Pending Ratio Y/Y
0,9/1/2017,1001.0,"Agawam, MA",NaN,213700.0,-0.0387,-0.0286,52.0,0.0612,-0.2571,...,0.0,248200.0,-0.0028,0.0973,53.0,0.0600,-0.2535,0.0192,-0.0012,0.0049
1,9/1/2017,1002.0,"Amherst, MA",NaN,358500.0,-0.0465,0.0913,110.0,-0.0678,-0.0351,...,NaN,444400.0,0.0018,0.2396,110.0,-0.0678,-0.0351,0.0000,0.0000,0.0000
2,9/1/2017,1005.0,"Barre, MA",NaN,251300.0,-0.0408,0.1100,24.0,0.1429,-0.1724,...,NaN,250900.0,-0.0260,0.0321,24.0,0.1429,-0.1724,0.0000,0.0000,0.0000
3,9/1/2017,1007.0,"Belchertown, MA",NaN,333600.0,-0.0039,0.1120,82.0,-0.0353,-0.0889,...,NaN,331500.0,-0.0140,0.0805,82.0,-0.0353,-0.0889,0.0000,0.0000,0.0000
4,9/1/2017,1008.0,"Blandford, MA",NaN,186200.0,0.0081,-0.0432,14.0,0.0000,-0.2222,...,0.0,257300.0,0.0235,-0.3675,15.0,0.0000,-0.2105,0.0714,0.0000,0.0159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974061,5/1/2012,99801.0,"Juneau, AK",NaN,323900.0,-0.0182,NaN,82.0,-0.0353,NaN,...,NaN,369400.0,-0.0196,NaN,82.0,-0.0353,NaN,0.0000,0.0000,NaN
974062,5/1/2012,99833.0,"Petersburg, AK",NaN,211500.0,-0.0140,NaN,19.0,0.0556,NaN,...,NaN,274900.0,-0.0115,NaN,19.0,0.0556,NaN,0.0000,0.0000,NaN
974063,5/1/2012,99835.0,"Sitka, AK",NaN,497500.0,0.0386,NaN,25.0,0.0417,NaN,...,NaN,693300.0,0.0622,NaN,25.0,0.0417,NaN,0.0000,0.0000,NaN
974064,5/1/2012,99901.0,"Ketchikan, AK",NaN,253800.0,0.0213,NaN,77.0,-0.0253,NaN,...,NaN,284000.0,0.0238,NaN,77.0,-0.0253,NaN,0.0000,0.0000,NaN


In [5]:
inventory_df = inventory[["Month", "ZipCode", "ZipName","Median Listing Price"]].copy().dropna()
inventory_df["ZipCode"]=inventory_df["ZipCode"].map(lambda x:int(x))
inventory_df


,Month,ZipCode,ZipName,Median Listing Price
0,9/1/2017,1001,"Agawam, MA",213700.0
1,9/1/2017,1002,"Amherst, MA",358500.0
2,9/1/2017,1005,"Barre, MA",251300.0
3,9/1/2017,1007,"Belchertown, MA",333600.0
4,9/1/2017,1008,"Blandford, MA",186200.0
...,...,...,...,...
974060,5/1/2012,99737,"Delta Junction, AK",225000.0
974061,5/1/2012,99801,"Juneau, AK",323900.0
974062,5/1/2012,99833,"Petersburg, AK",211500.0
974063,5/1/2012,99835,"Sitka, AK",497500.0


In [6]:
inv_df = inventory_df.groupby(["ZipCode", "ZipName"]).median()["Median Listing Price"].reset_index()
inv_df


,ZipCode,ZipName,Median Listing Price
0,1001,"Agawam, MA",194900.0
1,1002,"Amherst, MA",334700.0
2,1005,"Barre, MA",218200.0
3,1007,"Belchertown, MA",281200.0
4,1008,"Blandford, MA",249500.0
...,...,...,...
15037,99737,"Delta Junction, AK",195000.0
15038,99801,"Juneau, AK",351500.0
15039,99833,"Petersburg, AK",236600.0
15040,99835,"Sitka, AK",458800.0


In [7]:
merged_df = pd.merge(inv_df,pop_df, how="inner", on="ZipCode")
merged_df

,ZipCode,ZipName,Median Listing Price,population,minimum_age,maximum_age,gender
0,1001,"Agawam, MA",194900.0,299,62.0,64.0,male
1,1001,"Agawam, MA",194900.0,616,45.0,49.0,male
2,1001,"Agawam, MA",194900.0,363,75.0,79.0,female
3,1001,"Agawam, MA",194900.0,634,55.0,59.0,female
4,1001,"Agawam, MA",194900.0,275,67.0,69.0,female
...,...,...,...,...,...,...,...
476408,99901,"Ketchikan, AK",254600.0,244,22.0,24.0,female
476409,99901,"Ketchikan, AK",254600.0,432,30.0,34.0,male
476410,99901,"Ketchikan, AK",254600.0,58,85.0,NaN,male
476411,99901,"Ketchikan, AK",254600.0,467,40.0,44.0,male


In [8]:
merged_df.groupby("ZipCode").count()

,ZipName,Median Listing Price,population,minimum_age,maximum_age,gender
ZipCode,,,,,,
1001,32,32,32,31,31,32
1002,32,32,32,31,31,32
1005,31,31,31,29,29,31
1007,33,33,33,31,30,32
1008,36,36,36,36,34,36
...,...,...,...,...,...,...
99737,32,32,32,31,30,32
99801,28,28,28,25,24,27
99833,28,28,28,28,26,28


In [9]:
#maryland
maryland = pd.read_csv("resources/montgomery-county-of-maryland.csv")
maryland
#pop_df = Population.drop(columns=["geo_id"]).rename(columns={"zipcode":"ZipCode"})
maryland_crime_df = maryland[["Incident ID", "Offence Code", "CR Number", "Dispatch Date/Time",
                              "Victims", "Crime Name1", "Crime Name2",
                              "Crime Name3","Zip Code"]].copy().rename(columns={"Zip Code":"ZipCode"}).dropna()
maryland_crime_df["ZipCode"]=maryland_crime_df["ZipCode"].map(lambda x:int(x))
maryland_crime_df



C:\Users\glaud\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (1,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Incident ID,Offence Code,CR Number,Dispatch Date/Time,Victims,Crime Name1,Crime Name2,Crime Name3,ZipCode
0,201103138,5404,16053266,10/16/2016 23:09,1,Crime Against Society,Driving Under the Influence,DRIVING UNDER THE INFLUENCE LIQUOR,20904
2,201113980,1201,16066886,12/29/2016 3:05,1,Crime Against Property,Robbery,ROBBERY - BUSINESS-GUN,20814
3,201123049,3520,17007073,2/9/2017 10:32,1,Crime Against Society,Drug/Narcotic Violations,DRUGS - OPIUM OR DERIVATIVE - SELL,20901
4,201123422,9041,17011140,3/2/2017 16:18,1,Other,All Other Offenses,SUICIDE - COMPLETED,20832
5,201151115,2303,170528659,9/4/2017 17:47,1,Crime Against Property,Shoplifting,LARCENY - SHOPLIFTING,20876
...,...,...,...,...,...,...,...,...,...
167636,201249618,2203,190035467,7/25/2019 20:22,1,Crime Against Property,Burglary/Breaking and Entering,BURGLARY - FORCED ENTRY-NONRESIDENTIAL,20910
167637,201249619,2902,190035483,7/25/2019 21:50,1,Crime Against Property,Destruction/Damage/Vandalism of Property,DAMAGE PROPERTY - PRIVATE,20906
167638,201249635,2202,190035494,7/25/2019 22:54,1,Crime Against Property,Burglary/Breaking and Entering,BURGLARY - FORCED ENTRY-RESIDENTIAL,20904
167639,201250543,3615,190036640,8/1/2019 2:53,1,Crime Against Society,All Other Offenses,SEX OFFENSE - INDECENT EXPOSURE TO ADULT,20850


In [10]:
gb = maryland_crime_df.groupby(["ZipCode","Crime Name2"]).count()["Crime Name1"].reset_index()
#gb.pivot(index="ZipCode", columns="Crime Name2", values="Crime Name1")
gb

,ZipCode,Crime Name2,Crime Name1
0,6,False Pretenses/Swindle/Confidence Game,1
1,209,All Other Offenses,1
2,874,Drug/Narcotic Violations,1
3,2079,Arson,1
4,2080,Theft from Building,1
...,...,...,...
1888,29876,Drug/Narcotic Violations,1
1889,29876,Simple Assault,1
1890,29878,Burglary/Breaking and Entering,1
1891,29878,Theft From Motor Vehicle,1


In [11]:
merged_2_df = maryland_crime_df.merge(merged_df, how="inner", on="ZipCode").dropna()
merged_2_df





,Incident ID,Offence Code,CR Number,Dispatch Date/Time,Victims,Crime Name1,Crime Name2,Crime Name3,ZipCode,ZipName,Median Listing Price,population,minimum_age,maximum_age,gender
1,201103138,5404,16053266,10/16/2016 23:09,1,Crime Against Society,Driving Under the Influence,DRIVING UNDER THE INFLUENCE LIQUOR,20904,"Silver Spring, MD",399200.0,1200,22.0,24.0,female
2,201103138,5404,16053266,10/16/2016 23:09,1,Crime Against Society,Driving Under the Influence,DRIVING UNDER THE INFLUENCE LIQUOR,20904,"Silver Spring, MD",399200.0,2101,45.0,49.0,female
5,201103138,5404,16053266,10/16/2016 23:09,1,Crime Against Society,Driving Under the Influence,DRIVING UNDER THE INFLUENCE LIQUOR,20904,"Silver Spring, MD",399200.0,1979,35.0,39.0,female
6,201103138,5404,16053266,10/16/2016 23:09,1,Crime Against Society,Driving Under the Influence,DRIVING UNDER THE INFLUENCE LIQUOR,20904,"Silver Spring, MD",399200.0,2122,30.0,34.0,female
7,201103138,5404,16053266,10/16/2016 23:09,1,Crime Against Society,Driving Under the Influence,DRIVING UNDER THE INFLUENCE LIQUOR,20904,"Silver Spring, MD",399200.0,2124,50.0,54.0,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4469124,201206353,3532,180046489,9/16/2018 20:53,1,Crime Against Society,Drug/Narcotic Violations,DRUGS - COCAINE - POSSESS,20010,"Washington, DC",599800.0,694,50.0,54.0,female
4469125,201206353,3532,180046489,9/16/2018 20:53,1,Crime Against Society,Drug/Narcotic Violations,DRUGS - COCAINE - POSSESS,20010,"Washington, DC",599800.0,1327,22.0,24.0,female
4469126,201206353,3532,180046489,9/16/2018 20:53,1,Crime Against Society,Drug/Narcotic Violations,DRUGS - COCAINE - POSSESS,20010,"Washington, DC",599800.0,1235,22.0,24.0,male
4469128,201206353,3532,180046489,9/16/2018 20:53,1,Crime Against Society,Drug/Narcotic Violations,DRUGS - COCAINE - POSSESS,20010,"Washington, DC",599800.0,491,10.0,14.0,female


In [12]:
merged_2_df.groupby("ZipCode").count()

,Incident ID,Offence Code,CR Number,Dispatch Date/Time,Victims,Crime Name1,Crime Name2,Crime Name3,ZipName,Median Listing Price,population,minimum_age,maximum_age,gender
ZipCode,,,,,,,,,,,,,,
2852,24,24,24,24,24,24,24,24,24,24,24,24,24,24
2871,32,32,32,32,32,32,32,32,32,32,32,32,32,32
2904,27,27,27,27,27,27,27,27,27,27,27,27,27,27
2906,54,54,54,54,54,54,54,54,54,54,54,54,54,54
2910,31,31,31,31,31,31,31,31,31,31,31,31,31,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21703,32,32,32,32,32,32,32,32,32,32,32,32,32,32
21740,32,32,32,32,32,32,32,32,32,32,32,32,32,32
21771,1073,1073,1073,1073,1073,1073,1073,1073,1073,1073,1073,1073,1073,1073


In [13]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [14]:
# Define database and collection
db = client.etlproject_db
collection = db.realestate

In [15]:
#collection.insert_many(merged_df.to_dict("record"))
collection.insert_many(merged_2_df.to_dict("record"))